### Reliability across layers

In [ ]:
from pathlib import Path
import os
import sys
import random
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
import nibabel as nb
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from scipy.stats import gaussian_kde
from palettable.matplotlib import Inferno_20 as ColMap
from fmri_decoder.data import SurfaceData
from src.data import Data
from src.config import SESSION, DIR_BASE
from src.stats import permutation_test
plt.style.use(os.path.join(module_path, "src", "default.mplstyle"))

# set
SUBJ = "p1"
SESS = "VASO"
NSHUFFLE = 10000  # number of permutation iterations
FRAC = 0.1  # fraction of data used for permutation testing

In [ ]:
class Reliability:
    """Compute between-session correlation of a contrast sampled at a specific layer."""

    def __init__(self, subj=SUBJ, sess=SESS):
        self.subj = subj
        self.sess = sess

    def get_label(self):
        # get label and hemisphere
        data = Data(self.subj, [])
        surf_data = SurfaceData(data.file_layer, None, data.file_label)

        label_left = surf_data.load_label_intersection("lh")
        label_right = surf_data.load_label_intersection("rh")

        hemi = np.zeros(len(label_left)+len(label_right))
        hemi[len(label_left):] = 1
        label = np.append(label_left, label_right)

        return label, hemi

    def get_contrast(self, hemi, day, layer):
        file_in = (
            Path(DIR_BASE) 
            / self.subj 
            / "odc/results/Z/sampled" 
            / f"Z_all_left_right_{self.sess}{SESSION[self.subj][self.sess][day]}" 
            / f"{hemi}.Z_all_left_right_{self.sess}{SESSION[self.subj][self.sess][day]}_layer_{layer}.mgh"
        )
        return nb.load(file_in).get_fdata().flatten()

    def data(self, layer):
        label, hemi = self.get_label()
        data1 = np.zeros_like(hemi)
        data2 = np.zeros_like(hemi)
        data1[hemi == 0] = self.contrast("lh", 0, layer)[label[hemi == 0]]
        data1[hemi == 1] = self.contrast("rh", 0, layer)[label[hemi == 1]]
        data2[hemi == 0] = self.contrast("lh", 1, layer)[label[hemi == 0]]
        data2[hemi == 1] = self.contrast("rh", 1, layer)[label[hemi == 1]]

        # get subset of data for statistics
        ndata = int(FRAC * len(data1))
        ind = np.arange(len(data1))
        random.shuffle(ind)
        sample_0 = data1[ind[:ndata]]
        sample_1 = data2[ind[:ndata]]
        
        return sample_0, sample_1

    def corr(self, layer):
        sample_0, sample_1 = self.data(layer)
        return spearmanr(sample_0, sample_1)  # spearman


In [ ]:
rel = Reliability()